# Networking

### Michael Hund & Fernando Alfaro
_Formulario de feedback:_ https://goo.gl/forms/aFpTe9LGx9ObcMBu1

## ¿Que es?

Permite a un programa interactuar con otros, independiente si se encuentran en la misma máquina.

## ¿Cómo funciona?

- Ocupa un **medio**: *wifi*, *ethernet*, etc.

- Necesita **identificar** al otro programa: *dirección* y *puerto*.

- Consensuar un **protocolo**: *TCP*, *UDP* (reglas de la conexion).

## Dirección IP

Es la *etiqueta* que recibe una máquina en un medio para distinguirse.

### Versiones

- **IPv4**: número de 32 bits. Por ejemplo `192.30.253.112`

- **IPv6**: número de 128 bits. Por ejemplo `2001:0db8:85a3:0000:0000:8a2e:0370:7334`

## Puertos

Es la *compuerta* que utilizan los programas en un mismo ordenador para la recepción y envío de datos.

Se representan como un número de 16 bits. Un ordenador promedio posee 65.536 puertos.

## Protocolos

Son las *reglas* que rigen la comunicación entre los programas.

### TCP

- Permite enviar y recibir un *stream* de *bytes* de forma **confiable** (orden y fiabilidad).

- En caso de pérdida de información los datos son retransmitidos hasta una recepción correcta.

- Orientado a **conexión** (primero se conecta, luego se realiza la comunicación).

- Alta fiabilidad, alta latencia y encabezados pesados (mails, transferencias, etc).

### UDP

- Permite enviar y recibir un *datagramas* de forma **constante**.

- Hace su *mejor esfuerzo* para enviar la información, sacrificando su orden y fiabilidad.

- **No** orientado a **conexión**.

- Baja fiabilidad, alta latencia (audio/video streaming, live podcast, etc).

## Networking en Python

### Arquitectura Cliente-Servidor

![title](https://upload.wikimedia.org/wikipedia/commons/c/c9/Client-server-model.svg)
Fuente: [wikipedia](https://en.wikipedia.org/wiki/Client%E2%80%93server_model)

### Sockets

Un **socket** es un **objeto** encargado de manejar la comunicación entre los distintos programas.

Para definir un socket necesitamos:

**family**: versión de **dirección ip** a utilizar
    - IPv4: `AF_INET`
    - IPv6: `AF_INET6`

**type**: tipo de **protocolo** a utilizar
    - TCP: `SOCK_STREAM`
    - UDP: `SOCK_DGRAM`

In [0]:
# De esta forma el socket para una transmición IPv4 con TCP seria:
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

### Cliente TCP
Representa a una de las conexiones al servidor.

1. Se conecta a una dirección ip y puerto arbitrario.

2. Para el **envío** de información (siempre como *bytes*) utilizamos `sendall`.

3. Para la **recepción** desde el servidor utilizamos `recv` (con alguna potencia de 2 pequeña).

4. **Cerrar** la conexión mediante `close`

In [8]:
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

try:
    # Nos conectamos al servidor
    sock.connect(('python.org', 80))
    
    # Enviamos una solicitud
    sock.sendall('GET / HTTP/1.1\n\n\n'.encode('ascii'))
    
    # Recibimos respuesta
    data = sock.recv(4096)
    print(data.decode('ascii'))
    
except ConnectionError as e:
    print("Ocurrió un error.")
    
finally:
    # ¡No olvidemos cerrar la conexión!
    sock.close()

HTTP/1.1 301 Moved Permanently
Content-length: 0
Location: https:///




### Servidor TCP
Debe escuchar las conexiones que llegan y atender sus consultas.

1. **Enlazar** un socket a una dirección ip y puerto mediante `bind`.

2. **Escuchar** conexiones mediante `listen`.

3. **Aceptar** clientes y **manejarlos** mediante `accept`.

Al haber un número indefinidos clientes se recomienda crear un thread por cada conexión aceptada.

In [0]:
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Así podemos obtener el hostname de la máquina.
host = socket.gethostname()
port = 9001

# Nos enlazamos
sock.bind((host, port))

# Empezamos a escuchar
sock.listen()

counter = 0 # Solo por este ejemplo aceptaremos 5 conexiones individuales
while counter < 5:
    try:
        # Aceptamos a un cliente
        socket_cliente, address = sock.accept()
        print("Conexión aceptada desde", address)
        
        # Le enviamos un mensaje
        socket_cliente.sendall("Gracias por conectarte\n".encode("ascii"))
        
        # Lo desconectamos
        socket_cliente.close()
        counter += 1
        
    except ConnectionError:
        print("Ocurrió un error.")
        
# ¡No olvidemos cerrar la conexión!
sock.close()

A continuación un cliente acorde al servidor anterior:

In [0]:
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Obtenemos el nombre de la máquina a la que nos queremos conectar.
# En este caso, nos queremos conectar a esta misma máquina.
host = socket.gethostname()
port = 9001

# Nos conectamos
sock.connect((host, port))

# Recibimos el mensaje
data = sock.recv(4096)
print(data.decode("ascii"))

# ¡No olvidemos cerrar la conexión!
sock.close() 

### Cliente UDP

Debido a que no es necesaria una **conexión** podemos enviar directamente información a una dirección ip y puerto arbitrarios.

In [0]:
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

server_host = socket.gethostname()
server_port = 15000

mensaje = "Hola, simplemente te estoy enviando un mensaje.".encode('ascii')
sock.sendto(mensaje, (server_host, server_port))

# Opcionalmente podemos recibir un mensaje junto a su origen
data, (host_origen, puerto_origen) = sock.recvfrom(4096)
print(data.decode('utf-8'))

# ¡No olvidemos cerrar la conexión!
sock.close()

### Servidor UDP

La única diferencia es que nos debemos **enlazar** con la misma dirección ip y puerto anterior.

In [0]:
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# Con un string vacío como argumento del bind, indicamos que el socket es 
# alcanzable desde cualquier dirección que pueda tener el servidor.
sock.bind(("", 15000))

while True:
    data, (host_cliente, puerto_cliente) = sock.recvfrom(4096)
    print(data.decode('ascii'))
    respuesta = f"Aquí va mi respuesta para {host_cliente}."
    sock.sendto(respuesta.encode('utf-8'), (host_cliente, puerto_cliente))

## Ejemplo Cachipun 
Ahora veremos un ejemplo práctico de lo anterior, vayan a la carpeta ejemplo cachipun. Para correr el código deben ejecutar el Server y luego la Interfaz, esta última ejecútenla un par de veces para probar la interacción entre ventanas.

# Muchas gracias! 
## Éxito en este final de semestre :D!
### Respondan el feedback por favor, UwU